# Introduction to LDA and data cleaning
In this notebook, we introduce LDA and what we need for our model. We then proceed to load and clean a sample of the NOW corpus to fulfill our needs.

## What is LDA
[Latent Dirichlet Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a statistical model which we will use for topic modelling/discovery. LDA will, given a list of words belonging to a text, output the topics present and their probability. In here, a topic is represented as a probability distribution of words. Thus each text/document will be a distribution over the topics. In short, texts have an associated topic distribution and topics have a word distribution. 

The image below is the plate notation for LDA, where:
* θ<sub>m</sub> is the topic distribution for document m,
* φ<sub>k</sub> is the word distribution for topic k,
* z<sub>mn</sub> is the topic for the n-th word in document m, and
* w<sub>mn</sub> is the specific word.
* α is the parameter of the Dirichlet prior on the per-document topic distributions,
* β is the parameter of the Dirichlet prior on the per-topic word distribution,

![](LDA.png)

α and β are the parameters for the model. A big α means that documents are likely to be represented by a high number of topics and vice versa. Same goes for β, a high value meaning that topics are represented by a hign number of words. The number of topics that LDA outputs is dependent on our input and works a bit like clustering. If we allow too many topics we might end up splitting topics uselessly and a too few will make us group them unnecessarily. 

## The NOW corpus
This notebook shows the cleaning process that will be used for the ADA project. Here, only a sample of the data is used (from [here](https://www.corpusdata.org/now_corpus.asp)), but the methods should be the same once scaled to the full database available on the cluster.

The NOW database is composed of billions of words from online newspapers and magazines from 20 different countries. The data we downloaded comes in different files which can be used together or independently. These files are:

1. **now-samples-lexicon.txt**: this is the full dictionnary of the english language, a lexicon. It contains four clolumns, `wID` which is the word id, `word` the actual word, `lemma` which is family of the word (ie: if word is "walked", lemma is "walk") and `PoS` which is the part of speech.
2. **now-samples_sources.txt**: this is the source of every text, in order it contains the text id, the number of words, the date, the country, the website, the url and title of the article.
3. **text.txt**: this file has the complete texts of the articles, the first column is the `textID` in the format @@textID, the second column is the full text, complete with html paragraphs and headers. It is important to note that to prevent plagiarism, every 200 words, 10 words are replaced by the string "@ @ @ @ @ @ @ @ @ @". Combined words are also split, example "can't" is written as "ca n't" and punctuation is surrounded by spaces.
4. **wordLemPoS.txt**: finally, this file contains the `word`, `lemma` and `PoS` for each word in the texts, one by one, so one could read the texts by reading down the columns. Along with that is the `textID` from where the word is and an `ID (seq)` which is a unique indetifier for each word in the database. Each time a word is added this number is incremented.

## What we need from the NOW corpus for LDA
The model will take two inputs, a matrix with all the important words for each text, and a list of all the important words. By important, it is meant the words which will give us good topic modelling. For example, names, locations, simple words like "but, "I" or "and" will not give meaningfull results and are quite common in english (so-called stopwords). Other common words present in our database should be removed too. We also should use lemmas instead of words.

Therefore, the file `wordLemPoS.txt` (hence referred as wlp) is the most important here as it lists all the lemmas with their `textID` associated. Which means that with it we can lsit all the lemmas, remove those we do not want to make our word list, but also group them by texts to create our text-word matrix.

We will also need `now-sample_sources.txt` (hence referred as sources) to link the texts with the information we will deem useful. For example country, date or website.

These are thus the two file we will import and process here with the sample data but also those we will use with the data on the cluster.

## Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import findspark
findspark.init()

import re
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import DateType

from scipy.sparse import csr_matrix
from pyspark.ml.feature import *
from pyspark.sql.types import *

from pyspark.mllib.clustering import LDA, LDAModel

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Wlp processing


The goal of this part is to extract the useful data from wlp text files. Since they contain all the words of all the articles and the lemmas to replace them with.

In [2]:
#first read the text file
wlp_rdd = sc.textFile('sample_data/wordLem_poS.txt')

In [3]:
#the first 3 lines are useless headlines
header = wlp_rdd.take(3)

#so let's remove those headlines
noheaders = wlp_rdd.filter(lambda r: r != header[0])\
            .filter(lambda r: r != header[1])\
            .filter(lambda r: r != header[2])

In [4]:
#we split the elements separated by tabs
lines = noheaders.map(lambda r: r.split('\t'))

#identify the columns
wlp_schema = lines.map(lambda r: Row(textID=int(r[0]),idseq=int(r[1]),word=r[2],lemma=r[3],pos=r[4]))
wlp = spark.createDataFrame(wlp_schema)
wlp.show(5)

+----------+------+----+------+-------+
|     idseq| lemma| pos|textID|   word|
+----------+------+----+------+-------+
|1095362496|      |  fo| 11241|@@11241|
|1095362497|      |null| 11241|    <p>|
|1095362498|   sol| np1| 11241|    Sol|
|1095362499|yurick| np1| 11241| Yurick|
|1095362500|      |   ,| 11241|      ,|
+----------+------+----+------+-------+
only showing top 5 rows



### Word selection
It is very important to select the right words and the right number. The ocncept of "garbage in garbage out" has never been more true than with LDA. When we analyse a text we focus on certain words to extract it's meaning and topic. The same is true here since words like if, for, numbers, common names are not that useful.

Here, we provide and example of the process we will go through. However this is not really a data cleaning step as it will directly influence our model. It is more of a model preprocessing step. We will surely go through many iterations of this next part for our model to give the best results. 

First of all, we can remove all the words which have a PoS which do not interest us. For example number (`mc`,`mc1`,`m#`) or punctuation (`.`,`'`), etc...

Details:
1. `.`,`,`,`'` and `"` are punctuations
2. `null` are html tags from the websites
3. `mc`,`mc1` and `m#` are various numbers
3. `fo`, `fu` are random non sensical words

In [13]:
pos_remove = ['.',',',"\'",'\"','null','mc','mc1','m#','fo','fu']
wlp_nopos = wlp.filter(~wlp['pos'].isin(pos_remove)).drop('idseq','pos','word')

Now, we load our list of stopwords, the words that we are not going to use in LDA as they are too common or are common names. We can also remove the rows with no lemmas or those with lemmas that don't make sense or are not common enough.

In [14]:
#np.save('our_stopwords',stopwords)
stopwords = np.load('our_stopwords.npy').tolist()
print('Number of stopwords: ', len(stopwords))

Number of stopwords:  5639


In [15]:
#filter out stopwords and looking at the frequency of words without them
wlp_nostop = wlp_nopos.filter(~wlp['lemma'].isin(stopwords))
lemma_freq = wlp_nostop.groupBy('lemma').count().sort('count', ascending=False)
lemma_freq.show()

+----------+-----+
|     lemma|count|
+----------+-----+
|      year| 4272|
|      time| 3169|
|    people| 2913|
|      take| 2667|
|       use| 2244|
|      work| 2137|
|       day| 1819|
|     state| 1713|
|   company| 1698|
|   comment| 1667|
|      need| 1654|
|      want| 1579|
|      look| 1564|
|     world| 1553|
|government| 1551|
|      show| 1480|
|      give| 1480|
|   country| 1465|
|      find| 1464|
|     right| 1408|
+----------+-----+
only showing top 20 rows



We will also remove the most common and least common lemmas. These will be useless since they won't provide enough information for our LDA analysis. Here, we filter out the top 5% and bottom 10% of all lemmas.

<div class="alert alert-success">
Maybe should change from percentile to number for bottom filtering, depending on which one is the harshest (in here it is number)
</div>

In [16]:
#calculate percentiles and filtering out the lemmas above and below them
[bottom,top] = lemma_freq.approxQuantile('count', [0.1,0.99], 0.01)
bottom = 5
lemma_tokeep = lemma_freq.filter(lemma_freq['count']>bottom).filter(lemma_freq['count']<top)
print('Percentage of lemmas left: %.2f'%(lemma_tokeep.count()/lemma_freq.count()*100))

Percentage of lemmas left: 28.47


Making a inner join, we keep only the words which are in both lists! In the end, we can group the lemmas in their texts to create our text-word matrix.

In [24]:
#perform sql query and inner join
wlp_nostop.registerTempTable('wlp_nostop')
lemma_tokeep.registerTempTable('lemma_tokeep')

query = """
SELECT wlp_nostop.lemma, wlp_nostop.textID
FROM wlp_nostop
INNER JOIN lemma_tokeep ON wlp_nostop.lemma = lemma_tokeep.lemma
"""

wlp_kept = spark.sql(query)
wlp_bytext = wlp_kept.groupBy('textID').agg(collect_list('lemma'))\
                    .sort('textID')\
                    .withColumnRenamed('collect_list(lemma)','lemma_list')
wlp_bytext.show()

+------+--------------------+
|textID|          lemma_list|
+------+--------------------+
| 11241|[1970s, film, fil...|
| 11242|[online, happen, ...|
| 11243|[dough, dough, do...|
| 11244|[trail, launch, o...|
| 21242|[online, launch, ...|
| 21243|[recognize, indic...|
| 31240|[recognize, inten...|
| 31241|[online, online, ...|
| 31242|[settlement, sett...|
| 41240|[explain, hometow...|
| 41241|[scale, lack, pre...|
| 41244|[everyday, trail,...|
| 51243|[australia, austr...|
| 61240|[frustrate, inten...|
| 61242|[editor-in-chief,...|
| 71240|[indicator, requi...|
| 71241|[likelihood, requ...|
| 71242|[1970s, character...|
| 71243|[online, staff, s...|
| 71244|[bone, archaeolog...|
+------+--------------------+
only showing top 20 rows



## Sources
Contains all the additional informations about each text.

In [20]:
sources_rdd = sc.textFile('sample_data/now-samples-sources.txt')\
                .map(lambda r: r.split('\t'))

header = sources_rdd.take(3)
sources_rdd = sources_rdd.filter(lambda l: l != header[0])\
                .filter(lambda l: l != header[1])\
                .filter(lambda l: l != header[2])

In [21]:
#create schema and change data type for date
sources_schema = sources_rdd.map(lambda r: Row(textID=int(r[0]),nwords=int(r[1]),date=r[2],country=r[3],website=r[4],url=r[5],title=r[6],)) 
sources = spark.createDataFrame(sources_schema)
sources = sources.withColumn('date',to_date(sources.date, 'yy-MM-dd'))

In [22]:
sources.printSchema()

root
 |-- country: string (nullable = true)
 |-- date: date (nullable = true)
 |-- nwords: long (nullable = true)
 |-- textID: long (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- website: string (nullable = true)



In [23]:
sources.show(5)

+-------+----------+------+------+--------------------+--------------------+-------------------+
|country|      date|nwords|textID|               title|                 url|            website|
+-------+----------+------+------+--------------------+--------------------+-------------------+
|     US|2013-01-06|   397| 11241|Author of The War...|http://kotaku.com...|             Kotaku|
|     US|2013-01-06|   757| 11242|That's What They ...|http://michiganra...|     Michigan Radio|
|     US|2013-01-06|   755| 11243|Best of New York:...|http://www.nydail...|New York Daily News|
|     US|2013-01-06|  1677| 11244|Reflecting on a q...|http://www.oregon...|     OregonLive.com|
|     US|2013-01-11|   794| 21242|Ask Ars: Does Fac...|http://arstechnic...|       Ars Technica|
+-------+----------+------+------+--------------------+--------------------+-------------------+
only showing top 5 rows



## LDA
This is an expanded version on parrot's LDA using spark only.

In [45]:
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.ml.clustering import LDA, LDAModel
#from pyspark.mllib.linalg import Vectors

Can set certain filtering with CountVectorizer, not sure if that's useful since we already do that before hand.

In [36]:
cvmodel = CountVectorizer(inputCol="lemma_list", outputCol="raw_features").fit(wlp_bytext)
result_cv = cvmodel.transform(wlp_bytext)
result_cv.show()

+------+--------------------+--------------------+
|textID|          lemma_list|        raw_features|
+------+--------------------+--------------------+
| 11241|[1970s, film, fil...|(11399,[4,14,15,1...|
| 11242|[online, happen, ...|(11399,[0,1,11,16...|
| 11243|[dough, dough, do...|(11399,[0,2,3,4,5...|
| 11244|[trail, launch, o...|(11399,[0,2,4,5,6...|
| 21242|[online, launch, ...|(11399,[0,1,2,3,5...|
| 21243|[recognize, indic...|(11399,[3,4,5,6,7...|
| 31240|[recognize, inten...|(11399,[0,5,10,12...|
| 31241|[online, online, ...|(11399,[2,9,12,30...|
| 31242|[settlement, sett...|(11399,[6,8,36,50...|
| 41240|[explain, hometow...|(11399,[0,1,2,4,5...|
| 41241|[scale, lack, pre...|(11399,[1,3,4,11,...|
| 41244|[everyday, trail,...|(11399,[0,1,3,4,7...|
| 51243|[australia, austr...|(11399,[0,1,3,4,5...|
| 61240|[frustrate, inten...|(11399,[1,10,25,2...|
| 61242|[editor-in-chief,...|(11399,[0,1,2,7,9...|
| 71240|[indicator, requi...|(11399,[9,13,14,1...|
| 71241|[likelihood, requ...|(1

In [37]:
idfModel = IDF(inputCol="raw_features", outputCol="features").fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 
result_tfidf.show()

+------+--------------------+--------------------+--------------------+
|textID|          lemma_list|        raw_features|            features|
+------+--------------------+--------------------+--------------------+
| 11241|[1970s, film, fil...|(11399,[4,14,15,1...|(11399,[4,14,15,1...|
| 11242|[online, happen, ...|(11399,[0,1,11,16...|(11399,[0,1,11,16...|
| 11243|[dough, dough, do...|(11399,[0,2,3,4,5...|(11399,[0,2,3,4,5...|
| 11244|[trail, launch, o...|(11399,[0,2,4,5,6...|(11399,[0,2,4,5,6...|
| 21242|[online, launch, ...|(11399,[0,1,2,3,5...|(11399,[0,1,2,3,5...|
| 21243|[recognize, indic...|(11399,[3,4,5,6,7...|(11399,[3,4,5,6,7...|
| 31240|[recognize, inten...|(11399,[0,5,10,12...|(11399,[0,5,10,12...|
| 31241|[online, online, ...|(11399,[2,9,12,30...|(11399,[2,9,12,30...|
| 31242|[settlement, sett...|(11399,[6,8,36,50...|(11399,[6,8,36,50...|
| 41240|[explain, hometow...|(11399,[0,1,2,4,5...|(11399,[0,1,2,4,5...|
| 41241|[scale, lack, pre...|(11399,[1,3,4,11,...|(11399,[1,3,4,

In [52]:
%%time
lda_model = LDA(k=10, maxIter=10).fit(result_tfidf)

CPU times: user 230 ms, sys: 80.9 ms, total: 310 ms
Wall time: 36.2 s


In [58]:
#evaluate this for wider cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

In [117]:
# Describe topics.
topics = lda_model.describeTopics()
voc = np.array(cvmodel.vocabulary)
words_topics = topics.toPandas().termIndices

In [118]:
for t in range(len(words_topics)):
    print('Topic', t)
    print(voc[words_topics[t]])

Topic 0
['school' 'bc' 'patient' 'medical' 'education' 'courier' 'student'
 'doctor' 'anwar' 'village']
Topic 1
['pakistan' 'khan' 'oct' 'beer' 'penalty' 'salman' 'market' 'migrant'
 'camp' 'football']
Topic 2
['vegas' 'ball' 'cricket' 'pecos' 'shenandoah' 'la' 'league'
 'review-journal' 'match' 'liverpool']
Topic 3
['health' 'gas' 'cancer' 'antibiotic' 'hepatitis' 'animal' 'dr' 'island'
 'species' 'possum']
Topic 4
['game' 'team' 'race' 'bank' 'police' 'win' 'look' 'play' 'party' 'coach']
Topic 5
['people' 'tax' 'company' 'canada' 'want' 'public' 'use' 'drug' 'child'
 'government']
Topic 6
['market' 'quebec' 'porn' 'comment' 'trading' 'low' 'investor' 'industry'
 'content' 'product']
Topic 7
['*' 'muslim' 'police' 'app' 'share' 'harley-davidson' 'company' 'quarter'
 'islam' 'indian']
Topic 8
['dog' 'movie' 'city' 'film' 'artist' 'waterfront' 'bond' 'project'
 'refugee' 'zealand']
Topic 9
['spill' 'cork' 'military' 'boat' '*' 'syria' 'missile' 'russian'
 'aircraft' 'oil']


In [129]:
topic_distr = lda_model.transform(result_tfidf).drop('lemma_list','raw_features','features')

In [149]:
for t in topic_distr.rdd.map(lambda r: r[1]).collect():
    print(argmax(t))

8
4
8
5
5
7
2
5
5
5
3
8
5
1
0
6
5
8
0
9
7
3
6
5
3
4
5
5
3
4
8
5
5
4
4
2
4
3
5
3
8
8
4
5
4
5
4
5
5
4
4
6
0
4
5
4
4
4
9
5
5
5
6
6
4
5
4
3
5
8
4
4
4
2
5
4
5
5
5
5
5
8
5
5
9
4
4
5
5
5
5
6
5
3
6
5
5
7
5
3
5
3
5
5
4
5
5
4
8
5
5
5
5
5
4
3
5
5
4
5
4
4
8
5
5
4
5
5
3
2
4
4
5
5
5
4
7
4
6
3
5
7
5
3
4
9
5
3
5
3
4
8
4
5
5
5
8
7
4
5
4
5
4
1
9
7
5
6
5
9
7
5
6
7
6
5
1
1
1
5
5
5
7
5
4
8
4
4
2
6
5
5
5
5
8
5
4
5
1
5
4
5
8
5
5
4
6
4
5
6
5
5
5
8
8
5
4
5
5
7
5
4
7
6
5
5
4
5
4
5
5
9
4
5
5
4
5
8
7
4
8
0
4
4
4
4
5
4
5
5
2
5
4
5
5
5
7
4
5
5
4
8
6
4
5
4
4
4
7
9
5
4
5
4
3
5
5
5
5
5
3
5
2
3
4
5
5
4
5
9
3
5
4
7
5
9
5
0
1
5
4
5
5
5
5
4
6
1
5
5
5
5
6
7
5
5
5
6
7
4
2
4
4
4
4
6
5
5
5
5
5
4
8
5
5
4
5
4
3
5
5
5
5
7
4
6
5
0
4
6
5
3
4
0
8
5
5
5
7
4
8
5
6
5
8
4
4
5
4
4
5
4
5
7
7
5
9
2
3
4
4
4
4
4
5
7
7
6
5
4
5
3
7
8
9
5
3
3
5
5
3
7
8
4
4
4
5
9
5
4
4
5
5
5
4
9
7
7
9
4
4
4
5
1
4
1
5
6
7
5
5
7
6
5
5
8
5
3
4
5
5
4
5
4
5
8
8
8
4
2
6
4
9
4
4
5
4
4
8
5
4
5
8
3
5
5
4
5
5
0
5
5
5
4
8
5
6
4
4
8
4
5
5
0
5
4
4
4
7
7
6
8
5
9
8
4
5
5
8
9


In [154]:
from pyspark.sql.types import IntegerType
def argmax(x):
    return np.argmax(x)
argmax_udf = udf(lambda r: argmax(r), IntegerType())

In [156]:
main_topic = topic_distr.select('textID',argmax_udf('topicDistribution').alias('int_squared'))
main_topic.show()

Py4JJavaError: An error occurred while calling o12781.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 704.0 failed 1 times, most recent failure: Lost task 5.0 in stage 704.0 (TID 30716, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:86)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:85)
	at scala.collection.Iterator$$anon$12.nextCu